#### Creating Template Bank for Max M = 200, max q = 20, min M = 6
##### reduced nbanks from firstprecessing to 200

In [1]:
with open("config_file.ini", 'a') as fout:
    fout.write("""\
; Configuration file for pycbc_make_uberbank_workflow
;
; Tests and decisions for the choices made here are documented here:
;
;    https://www.lsc-group.phys.uwm.edu/ligovirgo/cbcnote/PyCBC/O2TemplateBank
;
; Template bank documentation can be found here:
;
;    http://pycbc.org/pycbc/latest/html/tmpltbank.html
;
; Documentation for the PyCBC workflow module is here:
;
;    http://pycbc.org/pycbc/latest/html/workflow.html

[common]
; Main options shared by different sections are gathered here

; Noise PSDs for geometric and sbank jobs
psd-txt = /home/isha.anantpurkar/PSDFiles/L1-AVERAGE_PSD-1162443617-100800.txt.gz
psd-xml = /home/isha.anantpurkar/PSDFiles/L1-AVERAGE_PSD-1162443617-100800.xml.gz

low-frequency-cutoff = 20

; Spin bounds
max-bh-spin = 0.9899
max-ns-spin = 0.05
nsbh-boundary = 2

[workflow]
; http://pycbc.org/pycbc/latest/html/workflow/initialization.html

; Store only "merged_triggers" this means only final result files, and all key
; intermediate products.
; file-retention-level = merged_triggers
file-retention-level = results

; Start and end times are needed to obey LIGO file naming conventions. Actual
; values are irrelevant. This can be overriden on the command line.
; Here we use the times matching the estimated PSD so we have an immediate
; reference on the resulting bank file name
start-time = 1000000000
end-time = 1000010000

[workflow-bbh]
; Options in here go only to the first sbank workflow (currently tagged BBH)

; num-cycles is how many repetitions of the sbank parallelization -> recombine
; process will be done
num-cycles = 13

; nbanks is how many parallel jobs to use, setting this large no longer causes
; overcoverage *but* be careful that the templates generated by all nbanks is
; not that large that it causes memory issues when readding. You can limit the
; number of templates added by each job and then run more total cycles. For
; weird parameter space (e.g. precession) set number of cycles to a large
; value, use a good number of nbanks, but accept only 10-100 templates per job
; This let's the bank slowly converge while you can monitor the periodic
; output!
nbanks = 200

; seed-bank can be used to provide an input to sbank jobs
; seed-bank = /path/to/file

; If using seed-bank, this will instruct sbank to use this to determine the
; initial chirp-mass bins. In this case a coarse job is not run
; use-seed-bank-for-chirp-bins=

; If not giving a seed-bank the coarse job will be used as a seed for the first
; parallel stage. Give this option and the coarse job will not be used as a
; seed. Then it is only used to determine initial chirp-mass bins
; do-not-use-coarse-job-as-seed =

[workflow-final]
; Options in here go only to the second sbank workflow (currently tagged FINAL)

; See above for option descriptions
num-cycles = 17
nbanks = 500

; Here the seed-bank is set automatically by the workflow generator.

; Not using seed bank for the chirp-mass bins as it does not cover the full
; space
; use-seed-bank-for-chirp-bins=

[workflow-ifos]
; Need to specify the active ifos for file naming.
h1 =
l1 =

[executables]
; http://pycbc.org/pycbc/latest/html/workflow/initialization.html

; All executables are listed here, will be equivalent to `which exe`
sbank = ${which:lalapps_cbc_sbank}
sbank_workflow = ${which:pycbc_make_sbank_workflow}
sbank_mchirp_bins = ${which:lalapps_cbc_sbank_hdf5_choose_mchirp_boundaries}
h5add = ${which:lalapps_cbc_sbank_hdf5_bankcombiner}
geom_aligned_bank = ${which:pycbc_geom_aligned_bank}

; Then options for all executables are added. These are added directly to the
; jobs as described here:
;    http://pycbc.org/pycbc/latest/html/workflow/initialization.html

[sbank_workflow]
; Options to the workflow generator

[llwadd]
; Global llwadd options, if any, go here

[geom_aligned_bank]
min-match = 0.97
random-seed = 1915
pn-order = threePointFivePN
f-low = ${common|low-frequency-cutoff}
f-upper = 1000
delta-f = 0.01
min-mass1 = 2.
max-mass1 = 5.
min-mass2 = 2.
max-mass2 = 5.
min-total-mass = 4.
max-total-mass = 10.
max-bh-spin-mag = ${common|max-bh-spin}
max-ns-spin-mag = ${common|max-ns-spin}
ns-bh-boundary-mass = ${common|nsbh-boundary}
psd-file = ${common|psd-txt}
split-bank-num = 75

[sbank_mchirp_bins]
; These options are sent to the sbank_mchirp_bins jobs
template-weight = equal

[sbank]
; These options are sent to *all* sbank jobs
; Options sent to specific sbank jobs are give below. There must be *no*
; conflicts here. Ie. if an option appears here it *cannot* be overridden in
; any of the sub-sections below.

; PSD-related options
reference-psd = ${common|psd-xml}
; This refers to the instrument in the reference-psd file.
instrument = H1L1
; F-high is lower than this if the waveform power doesn't get this high
fhigh-max = 2048.
flow = ${common|low-frequency-cutoff}
; Waveform
approximant = SEOBNRv2_ROM_DoubleSpin_HI
; Optimization choices
iterative-match-df-max = 8.0
cache-waveforms =

; Mass options common to sbank jobs
mass1-max = 99
mass2-max = 99
mtotal-max = 200
mratio-min = 1

; Spin options common to sbank jobs
; aligned-spin =
bh-spin-min = -${common|max-bh-spin}
bh-spin-max = ${common|max-bh-spin}
ns-spin-min = -${common|max-ns-spin}
ns-spin-max = ${common|max-ns-spin}
ns-bh-boundary-mass = ${common|nsbh-boundary}

[sbank-bbh]
; These options are sent to all sbank "BBH" jobs

; Mass/spin parameter options
mass1-min = 5.0
mass2-min = 5.0
mtotal-min = 10.0
mratio-max = 30.0

[sbank-final]
; These options are sent to all sbank "FINAL" jobs

; Mass/spin parameter options
mass1-min = 2.0
mass2-min = 2.0
; SEOBNRv2's eta = 0.01 limit
mratio-max = 97.989

[sbank-bbh-coarse]
; These options are sent only to the initial sbank-coarse job for "FINAL".
match-min = 0.98
convergence-threshold = 50
max-new-templates = 100000

[pegasus_profile-sbank-bbh-coarse]
; These are pegasus specific options for sbank jobs with the coarse tag.
; These will appear in the .sub files.
; FIXME: Need to verify this works with tags!

; This can take a lot of RAM! Other options for condor sub files can be
; specified in the same way. Things like accounting group are automatically
; added, as is get_env = True.
condor|request_memory = 10000

[sbank-final-coarse]
; These options are sent only to the initial sbank-coarse job for "BBH".
; This one is not a seed!!!
match-min = 0.90
convergence-threshold = 200
max-new-templates = 20000

[sbank-readder]
; These options are sent to all readder jobs

; Jobs should not terminate until all potential points are tried
convergence-threshold = 500000000
max-new-templates = 20000000000

[sbank-bbh-readder]
; These options are sent to all readder jobs for "BBH"
match-min = 0.98

[sbank-parallel-cycle0]
; parallel jobs, cycle0
convergence-threshold = 10
max-new-templates = 100

[sbank-parallel-cycle1]
; parallel jobs, cycle1
convergence-threshold = 10
max-new-templates = 100

[sbank-parallel-cycle2]
; parallel jobs, cycle2
convergence-threshold = 10
max-new-templates = 100

[sbank-parallel-cycle3]
; parallel jobs, cycle3
convergence-threshold = 10
max-new-templates = 200

[sbank-parallel-cycle4]
; parallel jobs, cycle4
convergence-threshold = 10
max-new-templates = 1000

[sbank-parallel-cycle5]
; parallel jobs, cycle5
convergence-threshold = 100
max-new-templates = 100

[sbank-parallel-cycle6]
; parallel jobs, cycle6
convergence-threshold = 100
max-new-templates = 100

[sbank-parallel-cycle7]
; parallel jobs, cycle7
convergence-threshold = 100
max-new-templates = 1000

[sbank-parallel-cycle8]
; parallel jobs, cycle8
convergence-threshold = 500
max-new-templates = 100

[sbank-parallel-cycle9]
; parallel jobs, cycle9
convergence-threshold = 500
max-new-templates = 1000

[sbank-parallel-cycle10]
; parallel jobs, cycle10
convergence-threshold = 1000
max-new-templates = 1000

[sbank-parallel-cycle11]
; parallel jobs, cycle11
convergence-threshold = 1000
; Don't expect max-new-templates to be reached here
max-new-templates = 1000000

[sbank-parallel-cycle12]
; parallel jobs, cycle12
convergence-threshold = 1000
max-new-templates = 1000000

[sbank-parallel-bbh]
; BBH parallel jobs

; All use the same match-min
match-min = 0.98

[sbank-final-cycle0&sbank-final-cycle1]
; BBH parallel and readder jobs, cycles 0 and 1
match-min = 0.965

[sbank-final-cycle2&sbank-final-cycle3]
; BBH parallel and readder jobs, cycles 2 and 3
match-min = 0.965

[sbank-final-cycle4&sbank-final-cycle5]
; BBH parallel and readder jobs, cycles 4 and 5
match-min = 0.965

[sbank-final-cycle6&sbank-final-cycle7]
; BBH parallel and readder jobs, cycles 6 and 7
match-min = 0.965

[sbank-final-cycle8&sbank-final-cycle9]
; BBH parallel and readder jobs, cycles 8 and 9
match-min = 0.965

[sbank-final-cycle10&sbank-final-cycle11&sbank-final-cycle12]
; BBH parallel and readder jobs, cycles 10 and 11 and 12
match-min = 0.965

[sbank-final-parallel-cycle13]
; These last cycles only used by "FINAL" intended to avoid any large holes
convergence-threshold = 5000
max-new-templates = 20000000
match-min = 0.95

[sbank-final-readder-cycle13&sbank-final-readder-cycle14&sbank-final-readder-cycle15&sbank-final-readder-cycle16]
; "FINAL" readder jobs at cycles 13 - 16
match-min = 0.95

[sbank-final-parallel-cycle14]
; These last cycles only used by "FINAL" intended to avoid any large holes
convergence-threshold = 5000
max-new-templates = 20000000
match-min = 0.95

[sbank-final-parallel-cycle15]
; These last cycles only used by "FINAL" intended to avoid any large holes
convergence-threshold = 50000
max-new-templates = 20000000
match-min = 0.95

[sbank-final-parallel-cycle16]
; These last cycles only used by "FINAL" intended to avoid any large holes
convergence-threshold = 50000
max-new-templates = 20000000
match-min = 0.95
""")